In [1]:
# imports 

import sys
import time
import os
import warnings

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

import numpy as np
import fitsio
from astropy.io import fits
import numpy.ma as ma

sys.path.insert(0, '/home/bekah/gphoton_working')
sys.path.insert(0, '/home/bekah/gphoton_working/gPhoton')

from photutils.aperture import CircularAperture
from photutils.utils import make_random_cmap
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

import pandas as pd 

from pyarrow import parquet

from gPhoton.parquet_utils import parquet_to_ndarray

from astropy.table import Table

from gPhoton.aspect import load_aspect_solution

from gPhoton.io.raw6 import load_raw6

from gPhoton.io._query.mast_query import retrieve_aspect
from gPhoton.io.mast import get_raw_paths 

In [ ]:
def centile_clip(image, centiles=(0, 90)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [11]:
# loading aspect table 
parq  = parquet.read_table('/home/bekah/gphoton_working/gPhoton/aspect/aspect.parquet')
aspect = parq.to_pandas()
aspect = aspect[aspect["eclipse"]==9869]

In [20]:
aspect = aspect.reset_index()
new_parq = pd.concat([df2,aspect.filter(['time','flags'])],axis=1,ignore_index=True)
new_parq

,0,1,2,3,4,5
0,160.707751,58.508319,-168.99505,9869,7.939713e+08,49
1,160.708994,58.508744,-169.00338,9869,7.939713e+08,49
2,160.709666,58.508729,-168.99197,9869,7.939713e+08,49
3,160.709739,58.508390,-168.96955,9869,7.939714e+08,49
4,160.710529,58.508866,-169.00638,9869,7.939714e+08,49
...,...,...,...,...,...,...
1636,160.691738,58.523320,-169.01203,9869,7.939730e+08,0
1637,160.690644,58.522796,-168.99629,9869,7.939730e+08,0
1638,160.689928,58.522097,-169.00924,9869,7.939730e+08,0
1639,160.689061,58.521748,-169.02865,9869,7.939730e+08,17


In [11]:
# loading aspect table to add time stamp and flags 
parq  = parquet.read_table('/home/bekah/gphoton_working/gPhoton/aspect/aspect.parquet')
aspect = parq.to_pandas()
aspect = aspect[aspect["eclipse"]==9869]
aspect = aspect.reset_index()

new_aspect_df = pd.read_csv("/home/bekah/gphoton_working/astrom_test_data/e09869/astrometry_xy_1s_09869")
df2 = new_aspect_df.filter(['frame','ra_center',
                                    'dec_center',
                                    'orientation'],
                                   axis=1)
df2['eclipse'] = 9869
# have to rename columns to ra, dec, and roll
# bc that's how og aspect parq is
df2 = df2.rename(columns={"ra_center": "ra",
                                  "dec_center": "dec",
                                  "orientation": "roll"})
df2 = df2.set_index('frame')
df2 = df2.reindex(range(len(aspect)), fill_value= 0)
df2 = df2.astype({'ra':'float64','dec':'float64','roll':'float64'})
df2 = df2.interpolate(method='linear', limit_direction='both', axis=0)

new_parq = pd.concat([df2,aspect.filter(['time','flags'])],axis=1)

# save to parquet 
new_parq.to_parquet('/home/bekah/gphoton_working/gPhoton/aspect/aspect2.parquet', compression=None)

In [12]:
len(new_parq)

1641

In [13]:
parq  = parquet.read_table('/home/bekah/gphoton_working/gPhoton/aspect/aspect2.parquet')
asp2 = parq.to_pandas()
asp2

,ra,dec,roll,eclipse,time,flags
0,160.707751,58.508319,-168.99505,9869,7.939713e+08,49
1,160.708994,58.508744,-169.00338,9869,7.939713e+08,49
2,160.709666,58.508729,-168.99197,9869,7.939713e+08,49
3,160.709739,58.508390,-168.96955,9869,7.939714e+08,49
4,160.710529,58.508866,-169.00638,9869,7.939714e+08,49
...,...,...,...,...,...,...
1636,160.691738,58.523320,-169.01203,9869,7.939730e+08,0
1637,160.690644,58.522796,-168.99629,9869,7.939730e+08,0
1638,160.689928,58.522097,-169.00924,9869,7.939730e+08,0
1639,160.689061,58.521748,-169.02865,9869,7.939730e+08,17


In [ ]:
%matplotlib notebook
plt.scatter(asp2['dec'],asp2["ra"])


In [ ]:
# path names
num_frames = 1400 

main_path = "/home/bekah/glcat_tests/"

xylist_folder = f"{main_path}xylists_9871/e09871/" #f'{main_path}xylists_lower_thresh/e09869_0.8_3/'
astrometry_temp = f'{main_path}astrometry_temp/'
    
# wcs and xylist paths
output_wcs = []
output_wcs_cropped = []
ver_wcs = []
fits_table = []
fits_table_cropped = []
    
for i in range(num_frames):
        frame_padded = str(i).zfill(5)
        # for feeding xylist to astrometry.net
        # think there's a fancier list comprehension way to do this but can't google bc on a plane
        fits_table.append(f'{xylist_folder}frame{i}.xyls')
        # for if we want to crop an area out of xylist by coordinates
        fits_table_cropped.append(f'{xylist_folder}frame{i}_crop.xyls')
        # for each frame, resulting new wcs
        output_wcs.append((i, f"{astrometry_temp}frame{i}.wcs"))
        # for cropped xylists wcs
        output_wcs_cropped.append((i, f"{astrometry_temp}frame{i}_crop.wcs"))
        
file_names = {"output_wcs": output_wcs,
                  "output_wcs_cropped": output_wcs_cropped}

In [ ]:
file_names['output_wcs_c']

In [ ]:
from typing import Optional
from pyarrow import parquet
import numpy as np
import os
import pandas as pd
import subprocess
import io
from astropy.io import fits

def make_refined_aspect_table(file_names, crop):
    """makes pandas df with wcs info for all frames, including failed ones."""
    # making an empty dataframe with column names, could pull col names from first
    # returned df but would get messy if first few frames failed
    asp_df = pd.DataFrame(columns=['frame', 'crpix0', 'crpix1', 'crval0', 'ra_tangent',
                                   'dec_tangent', 'pixx_tangent', 'pixy_tangent',
                                   'imagew', 'imageh', 'cd11', 'cd12', 'cd21', 'cd22',
                                   'det', 'parity', 'pixscale', 'orientation',
                                   'ra_center', 'dec_center', 'orientation_center',
                                   'ra_center_h', 'ra_center_m', 'ra_center_s',
                                   'dec_center_sign', 'dec_center_d', 'dec_center_m',
                                   'dec_center_s', 'ra_center_hms', 'dec_center_dms',
                                   'ra_center_merc', 'dec_center_merc', 'fieldarea',
                                   'fieldw', 'fieldh', 'fieldunits', 'decmin', 'decmax',
                                   'ramin', 'ramax', 'ra_min_merc', 'ra_max_merc',
                                   'dec_min_merc', 'dec_max_merc', 'merc_diff',
                                   'merczoom', 'failed_flag'])
    if crop:
        wcs_list = file_names["output_wcs_cropped"]
    else:
        wcs_list = file_names["output_wcs"]
    for frame, wcs_path in wcs_list:
        if os.path.exists(wcs_path):
            frame_wcs = get_aspect_from_wcsinfo(wcs_path)
            frame_wcs["frame"] = frame
            asp_df = pd.concat([asp_df, frame_wcs])
        if not os.path.exists(wcs_path):
            frame_no_wcs = pd.DataFrame(columns=['frame', 'failed_flag'])
            frame_no_wcs["frame"] = frame
            frame_no_wcs["failed_flag"] = True
            asp_df = pd.concat([asp_df, frame_no_wcs])
    asp_df = asp_df.set_index('frame')
    asp_df = asp_df.astype(
        {'crpix0': 'int64',
         'crpix1': 'int64',
         'crval0': 'float64',
         'ra_tangent': 'float64',
         'dec_tangent': 'float64',
         'pixx_tangent': 'int64',
         'pixy_tangent': 'int64',
         'imagew': 'int64',
         'imageh': 'int64',
         'cd11': 'float64',
         'cd12': 'float64',
         'cd21': 'float64',
         'cd22': 'float64',
         'det': 'float64',
         'parity': 'int64',
         'pixscale': 'float64',
         'orientation': 'float64',
         'ra_center': 'float64',
         'dec_center': 'float64',
         'orientation_center': 'float64',
         'ra_center_h': 'int64',
         'ra_center_m': 'int64',
         'ra_center_s': 'float64',
         'dec_center_sign': 'int64',
         'dec_center_d': 'int64',
         'dec_center_m': 'int64',
         'dec_center_s': 'float64',
         'ra_center_hms': 'O',
         'dec_center_dms': 'O',
         'ra_center_merc': 'float64',
         'dec_center_merc': 'float64',
         'fieldarea': 'float64',
         'fieldw': 'float64',
         'fieldh': 'float64',
         'fieldunits': 'O',
         'decmin': 'float64',
         'decmax': 'float64',
         'ramin': 'float64',
         'ramax': 'float64',
         'ra_min_merc': 'float64',
         'ra_max_merc': 'float64',
         'dec_min_merc': 'float64',
         'dec_max_merc': 'float64',
         'merc_diff': 'float64',
         'merczoom': 'int64'})
    # 'failed_flag': 'float64',
    #     'crval1': 'float64'
    # adds rows for failed frames
    asp_df = asp_df.reindex(range(1638), method=None)
    print("The list of failed frames is:")
    #null_list = asp_df.index[asp_df.isnull().all(1)].to_list()
    #null_df = {'failed_frames': null_list}
    #null_df = pd.DataFrame(null_df)
    #print(null_list)
    #null_df.to_csv(file_names["null_list"])
    # linearly interpolates
    #asp_df = asp_df.interpolate(method='linear', limit_direction='both', axis=0)
    return asp_df

def get_aspect_from_wcsinfo(wcs_path):
    """returns 1 row pd df with columns that are wcs values read from the
    astrometry.net output wcs file read w/ astrometry.net program wcsinfo,
     we use wcsinfo instead of reading the wcs header because it returns more
     information using other functions within astrometry.net (ie could
     recreate wcsinfo ourselves but it's a lot of work and written in c)."""
    output = subprocess.check_output(f"wcsinfo {wcs_path}",
                                     stderr=subprocess.STDOUT,
                                     shell=True)
    output = output.decode()  # bc was bytes not str
    output = "name value\n"+output
    frame_wcs = pd.read_csv(io.StringIO(output), sep=' ')
    frame_wcs = frame_wcs.set_index('name')
    frame_wcs = frame_wcs.transpose()
    return frame_wcs

In [ ]:
e9869_1s_cropped = make_refined_aspect_table(file_names, crop = True)

In [ ]:
e9869_1s_cropped.to_csv("/home/bekah/glcat_tests/aspect_solns/cropped_1s_large_crop_9871.csv")

In [ ]:
e9869_full = pd.read_csv("/home/bekah/glcat_tests/aspect_solns/e09869_aspect_soln_1s_thresh0.8_size3_dose_gaia_tycho")

In [ ]:
angle = 349-180
print(angle)

In [ ]:
%matplotlib notebook
plt.scatter(e9869_1s_cropped.index,e9869_1s_cropped["orientation"]+360,s=1)
plt.scatter(e9869_full.index,e9869_full["orientation"]+360,marker='+',c="green",s=1)
asprta_roll_9869 = aspect[aspect["eclipse"]==9869]["roll"].reset_index()
plt.scatter(asprta_roll_9869.index,asprta_roll_9869["roll"],s=1)

In [ ]:
e9869_image = fits.open('/home/bekah/gphoton_working/test_data/e09869/e09869-nd-t0001-b00-f0927-g_dose.fits')

from gPhoton.coadd import zero_flag_and_edge

In [ ]:
plt.imshow(centile_clip(e9869_image[0].data,(0,99.9)))

In [ ]:
e23456_dose = fits.open("/home/bekah/Downloads/e23456_nd_dose_test.fits")

In [ ]:
plt.imshow(e23456_dose[0].data)

In [ ]:
%matplotlib notebook
plt.scatter(e9869_1s_cropped["ra_center"],e9869_1s_cropped["dec_center"],s=.5,c='green')
#plt.scatter(e9869_full.index,e9869_full["orientation"]+360,marker='+',c="green",s=1)
#asprta_roll_9869 = aspect[aspect["eclipse"]==9871]["roll"].reset_index()
#plt.scatter(asprta_roll_9869.index,asprta_roll_9869["roll"],s=1)
plt.scatter(aspect[aspect["eclipse"]==9869]["ra"], aspect[aspect["eclipse"]==9869]["dec"],s=0.5,c='blue')
#plt.scatter(aspect[aspect["eclipse"]==9871]["ra"], aspect[aspect["eclipse"]==9871]["dec"],s=0.5,c='orange')